# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [24]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [25]:
weather_data = pd.read_csv("output_data/cities.csv")
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,Ucluelet,65,CA,1615004058,82,48.9329,-125.5528,46.99,4.00
1,Port Hardy,90,CA,1615004107,93,50.6996,-127.4199,42.80,18.41
2,Farafangana,100,MG,1615004332,87,-22.8167,47.8333,76.23,3.85
3,Hilo,90,US,1615004226,73,19.7297,-155.0900,71.60,6.91
4,Omboué,75,GA,1615004332,89,-1.5746,9.2618,77.54,8.14
...,...,...,...,...,...,...,...,...,...
561,Maïné Soroa,13,NE,1615004511,23,13.2114,12.0241,68.36,10.36
562,Severodvinsk,90,RU,1615004511,77,64.5635,39.8302,-4.00,3.98
563,Batagay-Alyta,0,RU,1615004511,74,67.8006,130.4114,-14.62,1.45
564,Erenhot,0,CN,1615004512,69,43.6475,111.9767,19.74,4.97


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [26]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Store latitude and longitude in locations
locations = weather_data[["Lat", "Lng"]]

# Store Humidity in humidity
humidity = weather_data["Humidity"]

In [27]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

In [28]:
max_intensity

100

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [29]:
# Narrow down the cities with wind speed less than 10 mph, cloudiness equals to 0 and max temp between 60 and 80
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()

narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
131,Nānpāra,0,IN,1615004370,19,27.8667,81.5000,74.34,4.45
155,Pisco,0,PE,1615004081,73,-13.7000,-76.2167,75.20,3.44
157,Chiang Khong,0,TH,1615004379,56,20.2613,100.4046,73.40,3.44
174,Chabahar,0,IR,1615004383,63,25.2919,60.6430,77.54,2.15
199,Mildura,0,AU,1615004222,19,-34.2000,142.1500,78.01,6.91
225,Makkah al Mukarramah,0,SA,1615004400,36,21.4267,39.8261,71.89,2.51
252,Salalah,0,OM,1615004412,60,17.0151,54.0924,71.60,2.30
269,Tessalit,0,ML,1615004417,22,20.1986,1.0114,70.41,9.24
372,Upington,0,ZA,1615004451,23,-28.4478,21.2561,70.03,8.61
418,Nicoya,0,CR,1615004466,57,10.1483,-85.4520,77.00,6.91


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [30]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
131,Nānpāra,IN,27.8667,81.5000,
155,Pisco,PE,-13.7000,-76.2167,
157,Chiang Khong,TH,20.2613,100.4046,
174,Chabahar,IR,25.2919,60.6430,
199,Mildura,AU,-34.2000,142.1500,
225,Makkah al Mukarramah,SA,21.4267,39.8261,
252,Salalah,OM,17.0151,54.0924,
269,Tessalit,ML,20.1986,1.0114,
372,Upington,ZA,-28.4478,21.2561,
418,Nicoya,CR,10.1483,-85.4520,


In [36]:
# Set parameters to search for a hotel
params = {
    "radius": 5000,
    "types": "lodging",
    "key": g_key
}

# Iterate through 
for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Lat"]
    lng = row["Lng"]
    
    params["location"] = f"{lat},{lng}"
    
    # Use the search term: "Hotel" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    name_address = requests.get(base_url, params=params)
    
    # convert to json
    name_address = name_address.json()
    
    # Grab the first hotel from the results and store the name
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["results"][0]["name"]
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")

hotel_df

Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.
Missing field/result... skipping.


,City,Country,Lat,Lng,Hotel Name
131,Nānpāra,IN,27.8667,81.5000,
155,Pisco,PE,-13.7000,-76.2167,
157,Chiang Khong,TH,20.2613,100.4046,
174,Chabahar,IR,25.2919,60.6430,
199,Mildura,AU,-34.2000,142.1500,
225,Makkah al Mukarramah,SA,21.4267,39.8261,
252,Salalah,OM,17.0151,54.0924,
269,Tessalit,ML,20.1986,1.0114,
372,Upington,ZA,-28.4478,21.2561,
418,Nicoya,CR,10.1483,-85.4520,


In [42]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]


In [43]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content = hotel_info)

# Add the layer to the map
fig.add_layer(markers)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))